# Experiments

In [67]:
import os
import IPython.display as ipd

for filename in os.listdir('./test_audio'):
    ipd.Audio('./test_audio/' + filename)

In [ ]:
import os
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf

for filename in os.listdir('./test'):
    data, sampling_rate = librosa.load('./test/' + filename)

    plt.figure(figsize=(12, 4))
    librosa.display.waveplot(data, sr=sampling_rate)

In [ ]:
import speech_recognition as sr

In [ ]:
r = sr.Recognizer()

In [ ]:
zero = sr.AudioFile('0_jackson_2.wav')

In [ ]:
with zero as source:
    audio = r.record(source)

In [ ]:
print(r.recognize_google(audio))

# This is the main training part:

## training with audio files:

In [1]:
#imports:

from export_model import *
from preprocess import *
import numpy as np
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

#from preprocess import *
#import keras
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
#from keras.utils import to_categorical
#from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
#from sklearn.preprocessing import LabelEncoder
#import itertools

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
#defaults at the end
feature_dim_1 = 20   #20
channel = 1          #1
epochs = 50         #50
batch_size = 100     #100
verbose = 1          #1
# change num_classes depending on the amount of labels
num_classes = 12

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Saving vectors of label - '9': 100%|█████████████████████████████████████████████████| 139/139 [00:02<00:00, 56.83it/s]


## model & prediction

In [3]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

## Train model:

In [4]:
model = get_model()
history = model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 1283 samples, validate on 856 samples
Epoch 1/50
1283/1283 [==============================] - ETA: 11s - loss: 7.9039 - acc: 0.03 - ETA: 6s - loss: 6.3143 - acc: 0.0300 - ETA: 4s - loss: 5.3269 - acc: 0.060 - ETA: 3s - loss: 4.7524 - acc: 0.070 - ETA: 2s - loss: 4.3514 - acc: 0.080 - ETA: 2s - loss: 4.0329 - acc: 0.098 - ETA: 1s - loss: 3.8077 - acc: 0.104 - ETA: 1s - loss: 3.6459 - acc: 0.110 - ETA: 0s - loss: 3.4838 - acc: 0.131 - ETA: 0s - loss: 3.3659 - acc: 0.143 - ETA: 0s - loss: 3.2968 - acc: 0.141 - ETA: 0s - loss: 3.2222 - acc: 0.145 - 3s 3ms/step - loss: 3.1492 - acc: 0.1567 - val_loss: 1.9228 - val_acc: 0.4439
Epoch 2/50
1283/1283 [==============================] - ETA: 1s - loss: 2.1278 - acc: 0.290 - ETA: 1s - loss: 2.1282 - acc: 0.260 - ETA: 1s - loss: 2.1642 - acc: 0.243 - ETA: 1s - loss: 2.1645 - acc: 0.245 - ETA: 1s - loss: 2.1262 - acc: 0.270 - ETA: 1s - loss: 2.0840 - acc: 0.285 - ETA: 0s - loss: 2.0578 - acc: 0.288 - ETA: 0s - loss: 2.0317 - acc: 0.301 - ET

1283/1283 [==============================] - ETA: 1s - loss: 0.2934 - acc: 0.880 - ETA: 1s - loss: 0.3623 - acc: 0.875 - ETA: 1s - loss: 0.3374 - acc: 0.873 - ETA: 1s - loss: 0.3177 - acc: 0.885 - ETA: 1s - loss: 0.3395 - acc: 0.868 - ETA: 1s - loss: 0.3535 - acc: 0.863 - ETA: 1s - loss: 0.3509 - acc: 0.865 - ETA: 0s - loss: 0.3481 - acc: 0.868 - ETA: 0s - loss: 0.3474 - acc: 0.868 - ETA: 0s - loss: 0.3330 - acc: 0.877 - ETA: 0s - loss: 0.3284 - acc: 0.880 - ETA: 0s - loss: 0.3128 - acc: 0.885 - 3s 2ms/step - loss: 0.3042 - acc: 0.8878 - val_loss: 0.2567 - val_acc: 0.8995
Epoch 15/50
1283/1283 [==============================] - ETA: 2s - loss: 0.2968 - acc: 0.910 - ETA: 1s - loss: 0.3002 - acc: 0.905 - ETA: 1s - loss: 0.2729 - acc: 0.916 - ETA: 1s - loss: 0.2639 - acc: 0.910 - ETA: 1s - loss: 0.2590 - acc: 0.904 - ETA: 1s - loss: 0.2687 - acc: 0.903 - ETA: 1s - loss: 0.2688 - acc: 0.902 - ETA: 0s - loss: 0.2712 - acc: 0.902 - ETA: 0s - loss: 0.2705 - acc: 0.904 - ETA: 0s - loss: 0.2633

Epoch 28/50
1283/1283 [==============================] - ETA: 1s - loss: 0.0784 - acc: 0.980 - ETA: 1s - loss: 0.0991 - acc: 0.975 - ETA: 1s - loss: 0.1044 - acc: 0.973 - ETA: 1s - loss: 0.1171 - acc: 0.967 - ETA: 1s - loss: 0.1082 - acc: 0.970 - ETA: 1s - loss: 0.1217 - acc: 0.961 - ETA: 0s - loss: 0.1107 - acc: 0.967 - ETA: 0s - loss: 0.1196 - acc: 0.962 - ETA: 0s - loss: 0.1252 - acc: 0.962 - ETA: 0s - loss: 0.1284 - acc: 0.960 - ETA: 0s - loss: 0.1247 - acc: 0.960 - ETA: 0s - loss: 0.1241 - acc: 0.959 - 2s 2ms/step - loss: 0.1241 - acc: 0.9595 - val_loss: 0.2580 - val_acc: 0.9089
Epoch 29/50
1283/1283 [==============================] - ETA: 1s - loss: 0.0910 - acc: 0.970 - ETA: 1s - loss: 0.1045 - acc: 0.965 - ETA: 1s - loss: 0.1201 - acc: 0.960 - ETA: 1s - loss: 0.1047 - acc: 0.970 - ETA: 1s - loss: 0.0944 - acc: 0.974 - ETA: 1s - loss: 0.1166 - acc: 0.963 - ETA: 0s - loss: 0.1267 - acc: 0.962 - ETA: 0s - loss: 0.1210 - acc: 0.965 - ETA: 0s - loss: 0.1173 - acc: 0.964 - ETA: 0s - 

1283/1283 [==============================] - ETA: 1s - loss: 0.0798 - acc: 0.980 - ETA: 1s - loss: 0.0515 - acc: 0.985 - ETA: 1s - loss: 0.0458 - acc: 0.986 - ETA: 1s - loss: 0.0654 - acc: 0.982 - ETA: 1s - loss: 0.0640 - acc: 0.982 - ETA: 1s - loss: 0.0579 - acc: 0.983 - ETA: 0s - loss: 0.0586 - acc: 0.981 - ETA: 0s - loss: 0.0571 - acc: 0.981 - ETA: 0s - loss: 0.0543 - acc: 0.982 - ETA: 0s - loss: 0.0550 - acc: 0.982 - ETA: 0s - loss: 0.0536 - acc: 0.982 - ETA: 0s - loss: 0.0596 - acc: 0.981 - 3s 2ms/step - loss: 0.0662 - acc: 0.9813 - val_loss: 0.3218 - val_acc: 0.9089
Epoch 42/50
1283/1283 [==============================] - ETA: 2s - loss: 0.0776 - acc: 0.990 - ETA: 2s - loss: 0.0876 - acc: 0.975 - ETA: 1s - loss: 0.0739 - acc: 0.980 - ETA: 1s - loss: 0.0836 - acc: 0.975 - ETA: 1s - loss: 0.0754 - acc: 0.978 - ETA: 1s - loss: 0.0663 - acc: 0.981 - ETA: 1s - loss: 0.0592 - acc: 0.984 - ETA: 0s - loss: 0.0582 - acc: 0.985 - ETA: 0s - loss: 0.0549 - acc: 0.986 - ETA: 0s - loss: 0.0553

## Export current model:

In [21]:
#export_model(model)

y_predicted = model.predict_classes(X_test, batch_size=batch_size)
y_true_val = np.argmax(y_test_hot,axis=1)

class_rep = classification_report(y_true_val,y_predicted,digits=5)

settings = {
    "feature_dim_1": feature_dim_1,
    "feature_dim_2": feature_dim_2,
    "channel": channel,
    "epochs": epochs,
    "batch_size": batch_size,
    "train_accuracy": str(history.history.get('acc')[-1]),
    "test_accuracy": str(history.history.get('val_acc')[-1]),
    "train_loss": str(history.history.get('loss')[-1]),
    "test_loss": str(history.history.get('val_loss')[-1]),
    "classification_report": class_rep,
}

print(export_model(model, settings))

Exported to folder ./models/20181212_15-44-46


## Import a model:

In [ ]:
# import_model(PATH)
imported_model = import_model("./models/xxx")

## Predict on a new file:

In [ ]:
print(predict('./test_audio/12.wav', model=model))

## Predict on a folder:

In [ ]:
FOLDER_PATH = './test_audio/'

for filename in os.listdir(FOLDER_PATH):
    pred = predict(FOLDER_PATH + filename, model=model)
    print(filename + " was predicted as: " + pred)

## Accuracy / Loss

In [ ]:
#Accuracy plot
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Full report with confusion matrix:

In [ ]:
#should import labels directly from folder:
labels, _, _= get_labels("./audio")
labArray = []
for label in labels:
    labArray.append(label)

le = LabelEncoder()

le.fit_transform(labArray)

full_multiclass_report(model, X_test, y_test_hot, classes=le.inverse_transform(np.arange(12)))

# Functions for all the metrics and results

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
def full_multiclass_report(model,
                           x,
                           y_true,
                           classes,
                           batch_size=32,
                           binary=False):

    # 1. Transform one-hot encoded y_true into their class number
    if not binary:
        y_true = np.argmax(y_true,axis=1)
    
    # 2. Predict classes and stores in y_pred
    y_pred = model.predict_classes(x, batch_size=batch_size)
    
    # 3. Print accuracy score
    print("Accuracy : "+ str(accuracy_score(y_true,y_pred)))
    
    print("")
    
    # 4. Print classification report
    print("Classification Report")
    print(classification_report(y_true,y_pred,digits=5))    
    
    # 5. Plot confusion matrix
    cnf_matrix = confusion_matrix(y_true,y_pred)
    print(cnf_matrix)
    plot_confusion_matrix(cnf_matrix,classes=classes)

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()